In [35]:
import site
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec


In [36]:
from gensim.models.keyedvectors import KeyedVectors

In [37]:
from keras.models import load_model


In [38]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
# from keras.models import Sequential, load_model, model_from_config

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from gensim.models.keyedvectors import KeyedVectors
# from keras import backend as K

In [39]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model[i])        
    vec = np.divide(vec,noOfWords)
    return vec

In [40]:
def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs

In [41]:
def sent2word(x):
    stop_words = set(stopwords.words('english')) 
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

In [42]:
# def convertToVec(text):
#     content=text
#     if len(content) > 20:
#         num_features = 300
#         model = KeyedVectors.load_word2vec_format("word2vecmodel.bin", binary=True)
#         clean_test_essays = []
#         clean_test_essays.append(sent2word(content))
#         testDataVecs = getVecs(clean_test_essays, model, num_features )
#         testDataVecs = np.array(testDataVecs)
#         testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

#         lstm_model = load_model("final_lstm.h5")
#         preds = lstm_model.predict(testDataVecs)
#         return str(round(preds[0][0]))

In [43]:
def convertToVec(text):
    content=text
    if len(content) > 20:
        num_features = 300
        model = KeyedVectors.load_word2vec_format("word2vecmodel.bin", binary=True)
        clean_test_essays = []
        clean_test_essays.append(sent2word(content))
        testDataVecs = getVecs(clean_test_essays, model, num_features )
        testDataVecs = np.array(testDataVecs)
        testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

        lstm_model = load_model("final_lstm.h5")
        bidirectional_model = load_model("final_bidirectional_lstm.h5")
        preds_lstm = lstm_model.predict(testDataVecs)
        preds_bidirectional = bidirectional_model.predict(testDataVecs)
        return [str(round(preds_lstm[0][0])),str(round(preds_bidirectional[0][0]))]

In [44]:
final_text = "I not agreed the people shuld d in the computer a lot and I going to said way. My first reason is because if you are a old man or your not going to d wit your kids and you going to have eyes problem so thas way you shuld in d in the computer a lot. My secon reason is the you going to have problem wit your wife if she found out the you are talking to another woman online because a lot of people they wife found out the they cant d wit other woman. My teard reason is the if you are a kid like me you not going to have fun and you not going to have friends because the computer problem came and go and not going to d wit your family and you lil sister out said playing wit her. So thas why you shulding d a lot in the computer."
score = convertToVec(final_text)

1/1 [==============================] - 1s 1s/step


In [45]:
print(score)
print("score predicted by lstm model",score[0])
print("score predicted by bidirectional model",score[1])

['5', '5']
score predicted by lstm model 5
score predicted by bidirectional model 5


In [46]:
def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

In [47]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model